In [73]:
import utilities
import pandas as pd
import os

If it's the first time we run the code, we'll download the data from bigquery and create a local copy

In [74]:
dataFileName = "data.json"
if (not os.path.exists(dataFileName)):
    df = utilities.retrieveBigQueryData("bigquery-public-data:google_analytics_sample.ga_sessions_20170801")
    df.to_json(dataFileName, orient="records")

Let's import the data

In [75]:
df = pd.read_json(dataFileName)

Let's check the size of the data

In [76]:
print("Rows: ", df.shape[0])
print("Columns: ", df.shape[1])

Rows:  2556
Columns:  16


Let's check the available columns, the types, the number of empty values

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   visitorId             0 non-null      float64
 1   visitNumber           2556 non-null   int64  
 2   visitId               2556 non-null   int64  
 3   visitStartTime        2556 non-null   int64  
 4   date                  2556 non-null   int64  
 5   totals                2556 non-null   object 
 6   trafficSource         2556 non-null   object 
 7   device                2556 non-null   object 
 8   geoNetwork            2556 non-null   object 
 9   customDimensions      2556 non-null   object 
 10  hits                  2556 non-null   object 
 11  fullVisitorId         2556 non-null   float64
 12  userId                0 non-null      float64
 13  clientId              0 non-null      float64
 14  channelGrouping       2556 non-null   object 
 15  socialEngagementType 

There are 3 empty columns, related to user id. This is probably due to privacy reasons, since this is a sample dataset.

Let's drop the data.

In [78]:
df.drop(columns=["visitorId", "userId", "clientId"], inplace=True)

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   visitNumber           2556 non-null   int64  
 1   visitId               2556 non-null   int64  
 2   visitStartTime        2556 non-null   int64  
 3   date                  2556 non-null   int64  
 4   totals                2556 non-null   object 
 5   trafficSource         2556 non-null   object 
 6   device                2556 non-null   object 
 7   geoNetwork            2556 non-null   object 
 8   customDimensions      2556 non-null   object 
 9   hits                  2556 non-null   object 
 10  fullVisitorId         2556 non-null   float64
 11  channelGrouping       2556 non-null   object 
 12  socialEngagementType  2556 non-null   object 
dtypes: float64(1), int64(4), object(8)
memory usage: 259.7+ KB


Let's see a sample of the data

In [80]:
pd.set_option('display.max_columns', None)

In [81]:
df.head(3)

,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,channelGrouping,socialEngagementType
0,1,1501583974,1501583974,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Car...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2.248282e+18,Organic Search,Not Socially Engaged
1,1,1501616585,1501616585,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",8.647436e+18,Organic Search,Not Socially Engaged
2,1,1501583344,1501583344,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2.055840e+18,Organic Search,Not Socially Engaged


We see some peculiar features of the data:

- the date column has an improper format
- some columns contain nested objects
- some columns contain lists

Let's clean the date column format. Let's check whethere there are multiple formats in the data

In [82]:
df.date.unique()

array([20170801], dtype=int64)

Let's transform the dates to strings, and add proper formatting

In [83]:
df.date = df.date.map(lambda val: str(val)).map(lambda val: val[0:4] + "-" + val[4:6] + "-" + val[6:8])
df.date.unique()

array(['2017-08-01'], dtype=object)

Let's work on the nested object data, by extracting each object property in a new column.

In [84]:
import utilities
from IPython.display import display

In [85]:
objCols = utilities.getListOfObjectColumns(df)

# Iterate the extraction process until there are no more object-type columns
while (len(objCols) > 0):

    # For each nested column, extract the data in new columns
    for col in objCols:
        print("Original data")
        display(df[[col]].head(3))

        print("Extracted data")
        utilities.extractNestedObjectData(df, col)
        extractedCols = [newcol for newcol in df.columns if newcol.startswith(col)]
        display(df[extractedCols].head(3))
        
        print()
    
    objCols = utilities.getListOfObjectColumns(df)

Original data


,totals
0,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time..."
1,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time..."
2,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time..."


Extracted data


,totals_visits,totals_hits,totals_pageviews,totals_timeOnSite,totals_bounces,totals_transactions,totals_transactionRevenue,totals_newVisits,totals_screenviews,totals_uniqueScreenviews,totals_timeOnScreen,totals_totalTransactionRevenue,totals_sessionQualityDim
0,1,1,1,NaN,1.0,NaN,NaN,1.0,None,None,None,NaN,1
1,1,1,1,NaN,1.0,NaN,NaN,1.0,None,None,None,NaN,1
2,1,1,1,NaN,1.0,NaN,NaN,1.0,None,None,None,NaN,1



Original data


...

,trafficSource_adwordsClickInfo_campaignId,trafficSource_adwordsClickInfo_adGroupId,trafficSource_adwordsClickInfo_creativeId,trafficSource_adwordsClickInfo_criteriaId,trafficSource_adwordsClickInfo_page,trafficSource_adwordsClickInfo_slot,trafficSource_adwordsClickInfo_criteriaParameters,trafficSource_adwordsClickInfo_gclId,trafficSource_adwordsClickInfo_customerId,trafficSource_adwordsClickInfo_adNetworkType,trafficSource_adwordsClickInfo_targetingCriteria,trafficSource_adwordsClickInfo_isVideoAd
0,None,None,None,None,NaN,None,not available in demo dataset,None,None,None,None,None
1,None,None,None,None,NaN,None,not available in demo dataset,None,None,None,None,None
2,None,None,None,None,NaN,None,not available in demo dataset,None,None,None,None,None



Original data


,trafficSource_adwordsClickInfo_targetingCriteria
0,None
1,None
2,None


Extracted data


,trafficSource_adwordsClickInfo_targetingCriteria_boomUserlistId
0,None
1,None
2,None


Let's see the new size of the dataset

In [86]:
print("Rows: ", df.shape[0])
print("Columns: ", df.shape[1])

Rows:  2556
Columns:  70


Let's check which columns are empty, and remove them

In [87]:
emptycols = []
for col in df.columns:
    if (df[col].isna().all()):
        emptycols.append(col)
print("Empty columns:", emptycols)

Empty columns: ['totals_screenviews', 'totals_uniqueScreenviews', 'totals_timeOnScreen', 'trafficSource_adwordsClickInfo_campaignId', 'trafficSource_adwordsClickInfo_adGroupId', 'trafficSource_adwordsClickInfo_creativeId', 'trafficSource_adwordsClickInfo_criteriaId', 'trafficSource_adwordsClickInfo_customerId', 'trafficSource_adwordsClickInfo_targetingCriteria_boomUserlistId', 'trafficSource_campaignCode', 'device_javaEnabled']


In [88]:
df.drop(columns=emptycols, inplace=True)

In [89]:
print("Rows: ", df.shape[0])
print("Columns: ", df.shape[1])

Rows:  2556
Columns:  59


Let's print a sample of the formatted dataset

In [90]:
df.head()

,visitNumber,visitId,visitStartTime,date,totals_visits,totals_hits,totals_pageviews,totals_timeOnSite,totals_bounces,totals_transactions,totals_transactionRevenue,totals_newVisits,totals_totalTransactionRevenue,totals_sessionQualityDim,trafficSource_referralPath,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_adContent,trafficSource_adwordsClickInfo_page,trafficSource_adwordsClickInfo_slot,trafficSource_adwordsClickInfo_criteriaParameters,trafficSource_adwordsClickInfo_gclId,trafficSource_adwordsClickInfo_adNetworkType,trafficSource_adwordsClickInfo_isVideoAd,trafficSource_isTrueDirect,device_browser,device_browserVersion,device_browserSize,device_operatingSystem,device_operatingSystemVersion,device_isMobile,device_mobileDeviceBranding,device_mobileDeviceModel,device_mobileInputSelector,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_flashVersion,device_language,device_screenColors,device_screenResolution,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_cityId,geoNetwork_networkDomain,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_networkLocation,customDimensions,hits,fullVisitorId,channelGrouping,socialEngagementType
0,1,1501583974,1501583974,2017-08-01,1,1,1,NaN,1.0,NaN,NaN,1.0,NaN,1,None,(not set),(direct),(none),None,None,NaN,None,not available in demo dataset,None,None,None,None,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile,Americas,Caribbean,St. Lucia,(not set),(not set),(not set),not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset,[],"[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2.248282e+18,Organic Search,Not Socially Engaged
1,1,1501616585,1501616585,2017-08-01,1,1,1,NaN,1.0,NaN,NaN,1.0,NaN,1,None,(not set),(direct),(none),None,None,NaN,None,not available in demo dataset,None,None,None,None,Chrome,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,"[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",8.647436e+18,Organic Search,Not Socially Engaged
2,1,1501583344,1501583344,2017-08-01,1,1,1,NaN,1.0,NaN,NaN,1.0,NaN,1,None,(not set),(direct),(none),None,None,NaN,None,not available in demo dataset,None,None,None,None,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile,Asia,Southern Asia,India,Delhi,(not set),New Delhi,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset,"[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2.055840e+18,Organic Search,Not Socially En

In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 59 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   visitNumber                                        2556 non-null   int64  
 1   visitId                                            2556 non-null   int64  
 2   visitStartTime                                     2556 non-null   int64  
 3   date                                               2556 non-null   object 
 4   totals_visits                                      2556 non-null   int64  
 5   totals_hits                                        2556 non-null   int64  
 6   totals_pageviews                                   2556 non-null   int64  
 7   totals_timeOnSite                                  1310 non-null   float64
 8   totals_bounces                                     1238 non-null   float64
 9   totals_t

Let's move on to extracting the data in the array-type columns

In [92]:
arrayCols = utilities.getListOfArrayColumns(df)
arrayCols

['customDimensions', 'hits']

In [93]:
df[arrayCols].head()

,customDimensions,hits
0,[],"[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut..."
1,"[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu..."
2,"[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut..."
3,"[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 0, 'minut..."
4,"[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 22, 'minu..."


Let's check the number of values per array in each column

In [94]:
for col in arrayCols:
    print(col)
    lengths = df[col].map(lambda list: 0 if list == None else len(list)).unique()
    print(lengths, "\n")

customDimensions
[0 1] 

hits
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  32  33  34  35  36  37
  38  39  42  44  45  47  48  53  55  58  60  62  63  64  71  78  80  81
  82  84 100 140 156 193 302] 



Each cell in the ```custom``` contains either 0 or 1 object value, so we can simply extract the object in it and read its' nested data.

In [95]:
df.customDimensions = df.customDimensions.map(lambda list: None if (list == None or len(list) == 0) else list[0])

In [96]:
df[["customDimensions"]].head()

,customDimensions
0,None
1,"{'index': 4, 'value': 'North America'}"
2,"{'index': 4, 'value': 'APAC'}"
3,"{'index': 4, 'value': 'EMEA'}"
4,"{'index': 4, 'value': 'North America'}"


In [97]:
utilities.extractNestedObjectData(df, "customDimensions")

In [98]:
df.head()

,visitNumber,visitId,visitStartTime,date,totals_visits,totals_hits,totals_pageviews,totals_timeOnSite,totals_bounces,totals_transactions,totals_transactionRevenue,totals_newVisits,totals_totalTransactionRevenue,totals_sessionQualityDim,trafficSource_referralPath,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_adContent,trafficSource_adwordsClickInfo_page,trafficSource_adwordsClickInfo_slot,trafficSource_adwordsClickInfo_criteriaParameters,trafficSource_adwordsClickInfo_gclId,trafficSource_adwordsClickInfo_adNetworkType,trafficSource_adwordsClickInfo_isVideoAd,trafficSource_isTrueDirect,device_browser,device_browserVersion,device_browserSize,device_operatingSystem,device_operatingSystemVersion,device_isMobile,device_mobileDeviceBranding,device_mobileDeviceModel,device_mobileInputSelector,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_flashVersion,device_language,device_screenColors,device_screenResolution,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_cityId,geoNetwork_networkDomain,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_networkLocation,customDimensions_index,customDimensions_value,hits,fullVisitorId,channelGrouping,socialEngagementType
0,1,1501583974,1501583974,2017-08-01,1,1,1,NaN,1.0,NaN,NaN,1.0,NaN,1,None,(not set),(direct),(none),None,None,NaN,None,not available in demo dataset,None,None,None,None,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile,Americas,Caribbean,St. Lucia,(not set),(not set),(not set),not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset,NaN,None,"[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2.248282e+18,Organic Search,Not Socially Engaged
1,1,1501616585,1501616585,2017-08-01,1,1,1,NaN,1.0,NaN,NaN,1.0,NaN,1,None,(not set),(direct),(none),None,None,NaN,None,not available in demo dataset,None,None,None,None,Chrome,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,4.0,North America,"[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",8.647436e+18,Organic Search,Not Socially Engaged
2,1,1501583344,1501583344,2017-08-01,1,1,1,NaN,1.0,NaN,NaN,1.0,NaN,1,None,(not set),(direct),(none),None,None,NaN,None,not available in demo dataset,None,None,None,None,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile,Asia,Southern Asia,India,Delhi,(not set),New Delhi,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset,4.0,APAC,"[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2.055840e+18,Organic Search,Not Socially Engaged
3,1,15015

For the ```hits``` column we'll need to export the data to a separate dataset, otherwise to extract it we'd multiply the number of rows in the original dataset.

Let's add an ID column to the base dataset, to link it with the hits dataset.

In [99]:
df = df.reset_index().rename(columns={"index": "id"})
df.head()

,id,visitNumber,visitId,visitStartTime,date,totals_visits,totals_hits,totals_pageviews,totals_timeOnSite,totals_bounces,totals_transactions,totals_transactionRevenue,totals_newVisits,totals_totalTransactionRevenue,totals_sessionQualityDim,trafficSource_referralPath,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_adContent,trafficSource_adwordsClickInfo_page,trafficSource_adwordsClickInfo_slot,trafficSource_adwordsClickInfo_criteriaParameters,trafficSource_adwordsClickInfo_gclId,trafficSource_adwordsClickInfo_adNetworkType,trafficSource_adwordsClickInfo_isVideoAd,trafficSource_isTrueDirect,device_browser,device_browserVersion,device_browserSize,device_operatingSystem,device_operatingSystemVersion,device_isMobile,device_mobileDeviceBranding,device_mobileDeviceModel,device_mobileInputSelector,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_flashVersion,device_language,device_screenColors,device_screenResolution,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_cityId,geoNetwork_networkDomain,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_networkLocation,customDimensions_index,customDimensions_value,hits,fullVisitorId,channelGrouping,socialEngagementType
0,0,1,1501583974,1501583974,2017-08-01,1,1,1,NaN,1.0,NaN,NaN,1.0,NaN,1,None,(not set),(direct),(none),None,None,NaN,None,not available in demo dataset,None,None,None,None,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile,Americas,Caribbean,St. Lucia,(not set),(not set),(not set),not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset,NaN,None,"[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2.248282e+18,Organic Search,Not Socially Engaged
1,1,1,1501616585,1501616585,2017-08-01,1,1,1,NaN,1.0,NaN,NaN,1.0,NaN,1,None,(not set),(direct),(none),None,None,NaN,None,not available in demo dataset,None,None,None,None,Chrome,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,4.0,North America,"[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",8.647436e+18,Organic Search,Not Socially Engaged
2,2,1,1501583344,1501583344,2017-08-01,1,1,1,NaN,1.0,NaN,NaN,1.0,NaN,1,None,(not set),(direct),(none),None,None,NaN,None,not available in demo dataset,None,None,None,None,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile,Asia,Southern Asia,India,Delhi,(not set),New Delhi,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset,4.0,APAC,"[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2.055840e+18,Organic Search,Not Socially Engaged


In [100]:
hitsdf = df[["id", "hits"]].rename(columns={"id": "session_id"})
hitsdf.head()

,session_id,hits
0,0,"[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut..."
1,1,"[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu..."
2,2,"[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut..."
3,3,"[{'hitNumber': 1, 'time': 0, 'hour': 0, 'minut..."
4,4,"[{'hitNumber': 1, 'time': 0, 'hour': 22, 'minu..."


In [101]:
df.drop(columns="hits", inplace=True)

Now that we've flattened and formatted the base dataset, let's perform further exploration.

Let's display some descriptive statistics.

In [102]:
df.describe(include="all")

,id,visitNumber,visitId,visitStartTime,date,totals_visits,totals_hits,totals_pageviews,totals_timeOnSite,totals_bounces,totals_transactions,totals_transactionRevenue,totals_newVisits,totals_totalTransactionRevenue,totals_sessionQualityDim,trafficSource_referralPath,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_adContent,trafficSource_adwordsClickInfo_page,trafficSource_adwordsClickInfo_slot,trafficSource_adwordsClickInfo_criteriaParameters,trafficSource_adwordsClickInfo_gclId,trafficSource_adwordsClickInfo_adNetworkType,trafficSource_adwordsClickInfo_isVideoAd,trafficSource_isTrueDirect,device_browser,device_browserVersion,device_browserSize,device_operatingSystem,device_operatingSystemVersion,device_isMobile,device_mobileDeviceBranding,device_mobileDeviceModel,device_mobileInputSelector,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_flashVersion,device_language,device_screenColors,device_screenResolution,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_cityId,geoNetwork_networkDomain,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_networkLocation,customDimensions_index,customDimensions_value,fullVisitorId,channelGrouping,socialEngagementType
count,2556.000000,2556.000000,2.556000e+03,2.556000e+03,2556,2556.0,2556.000000,2556.000000,1310.000000,1238.0,43.000000,4.300000e+01,1872.0,4.300000e+01,2556.000000,649,2556,2556,2556,5,12,58.0,58,2556,58,58,58,872,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2556,2210.0,2210,2.556000e+03,2556,2556
unique,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88,2,35,5,3,2,NaN,1,1,52,1,1,1,15,1,1,9,1,2,1,1,1,1,1,1,1,1,1,3,6,19,94,42,19,56,1,524,1,1,1,NaN,5,NaN,7,1
top,NaN,NaN,NaN,NaN,2017-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/,(not set),(direct),(none),(not provided),Display Ad created 3/11/14,NaN,Top,not available in demo dataset,CjwKCAjwzYDMBRA1EiwAwCv6Jt0kCNNM8qAVQ6PaGsphV9...,Google Search,False,True,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,NaN,North America,NaN,Organic Search,Not Socially Engaged
freq,NaN,NaN,NaN,NaN,2556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330,2504,2166,2166,3,11,NaN,58,2556,4,58,58,872,1900,2556,2556,834,2556,1741,2556,2556,2556,2556,2556,2556,2556,2556,2556,1742,1472,1372,1287,1388,1388,1388,2556,1003,2556,2556,2556,NaN,1372,NaN,1346,2556
mean,1277.500000,2.661189,1.501614e+09,1.501614e+09,NaN,1.0,5.177230,4.279734,325.051908,1.0,1.046512,1.931381e+08,1.0,2.067442e+08,5.832942,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.549451e+18,NaN,NaN
std,737.997967,12.022200,2.280455e+04,2.280423e+04,NaN,0.0,11.816349,7.966343,592.411724,0.0,0.213083,4.787008e+08,0.0,4.785247e+08,16.463750,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,3.107969e+18,NaN,NaN
min,0.000000,1.000000,1.501570e+09,1.501571e+09,NaN,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.990000e+06,1.0,2.990000e+06,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na